In [2]:
!pip install datasets transformers pandas evaluate

# Step 2: Import Libraries
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import evaluate
import pandas as pd
from google.colab import files

# Step 3: Upload Training Data
print("Please upload your training dataset (CSV file).")
uploaded_train = files.upload()

# Load the uploaded training data
train_file_name = list(uploaded_train.keys())[0]
print(f"Training data file '{train_file_name}' uploaded.")
train_data = pd.read_csv(train_file_name)

# Step 4: Preprocess Training Data
# Filter columns and handle missing values
train_data = train_data[['def_text', 'VesselGroup', 'age', 'predicted_severity']].dropna()

# Map severity levels to numerical values
severity_mapping = {"Low": 0, "Medium": 1, "High": 2, "Not a deficiency": 3}
train_data['labels'] = train_data['predicted_severity'].map(severity_mapping)

# Convert to Hugging Face dataset
dataset = Dataset.from_pandas(train_data)

# Tokenizer Setup
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# DistilBERT Model is used due to its efficiency and lower resource requirement while still being able complete tasks at a high performance
def tokenize_function(example):
    return tokenizer(
        example["def_text"],
        padding="max_length",
        truncation=True,
        max_length=128
    )

# Tokenize the dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Step 5: Split Dataset for Training and Evaluation
split_dataset = tokenized_dataset.train_test_split(test_size=0.2)
train_data = split_dataset["train"]
eval_data = split_dataset["test"]

# Step 6: Load Pretrained Model for Sequence Classification
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(severity_mapping)
)

# Step 7: Define Training Arguments
training_args = TrainingArguments(
    output_dir="./results",           # Directory for saving model checkpoints
    evaluation_strategy="steps",     # Save and evaluate periodically
    save_strategy="steps",
    save_steps=500,                  # Save every 500 steps
    learning_rate=2e-5,              # Learning rate
    per_device_train_batch_size=16,  # Batch size
    num_train_epochs=5,              # Number of epochs
    logging_dir="./logs",            # Logging directory
    logging_steps=50,                # Log every 50 steps
    load_best_model_at_end=True      # Automatically load the best model
)

# Step 8: Define Metrics
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# Step 9: Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=eval_data,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Step 10: Train the Model
trainer.train()

# Step 11: Save the Model
trainer.save_model("./trained_model")
tokenizer.save_pretrained("./trained_model")

print("Model training complete. Model and tokenizer saved to './trained_model'.")

# Step 12: Upload Test Data
print("Please upload your test dataset (CSV file).")
uploaded_test = files.upload()

# Load the uploaded test data
test_file_name = list(uploaded_test.keys())[0]
print(f"Test data file '{test_file_name}' uploaded.")
test_data = pd.read_csv(test_file_name)

# Process Test Data: Tokenize and Predict
test_data['def_text'] = test_data['def_text'].fillna("")  # Ensure no missing values in 'def_text'

# Tokenize the test data
test_dataset = Dataset.from_pandas(test_data)

tokenized_test = test_dataset.map(
    lambda x: tokenizer(x['def_text'], padding="max_length", truncation=True, max_length=128),
    batched=True
)

# Generate Predictions
predictions = trainer.predict(tokenized_test)

# Convert predictions to severity levels
predicted_classes = predictions.predictions.argmax(axis=-1)
predicted_severity = [list(severity_mapping.keys())[cls] for cls in predicted_classes]


# Add predicted_severity to the test data
test_data['predicted_severity'] = predicted_severity

# Keep only requested columns
output_test_data = test_data[['PscInspectionId', 'deficiency_code', 'predicted_severity']]

# Save the modified test data
output_file_name = "modified_test_data.csv"
output_test_data.to_csv(output_file_name, index=False)
print(f"Modified test data saved to '{output_file_name}'.")

# Step 13: Download the Modified Test Data
files.download(output_file_name)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
Please upload your training dataset (CSV file).


Saving resolved_predicted_severity.csv to resolved_predicted_severity.csv
Training data file 'resolved_predicted_severity.csv' uploaded.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

...

Saving psc_severity_test.csv to psc_severity_test.csv
Test data file 'psc_severity_test.csv' uploaded.


Map:   0%|          | 0/1101 [00:00<?, ? examples/s]

Modified test data saved to 'modified_test_data.csv'.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>